In [1]:
import pandas as pd
import re
from tqdm import tqdm
from ast import literal_eval
from urllib.parse import urlparse

In [4]:
df = pd.read_csv('')

In [5]:
df

,url
0,http://1337x.to/torrent/1048648/American-Snipe...
1,http://1337x.to/torrent/1110018/Blackhat-2015-...
2,http://1337x.to/torrent/1122940/Blackhat-2015-...
3,http://1337x.to/torrent/1124395/Fast-and-Furio...
4,http://1337x.to/torrent/1145504/Avengers-Age-o...
...,...
35373,https://lastpass.com/signup2.php?ac=1&from_uri...
35374,https://lastpass.com/signup2.php?ac=1&from_uri...
35375,https://lastpass.com/signup2.php?ac=1&from_uri...
35376,https://lastpass.com/signup2.php?ac=1&from_uri...


In [6]:
df['is_affiliate'] = 0

In [7]:
df

,url,is_affiliate
0,http://1337x.to/torrent/1048648/American-Snipe...,0
1,http://1337x.to/torrent/1110018/Blackhat-2015-...,0
2,http://1337x.to/torrent/1122940/Blackhat-2015-...,0
3,http://1337x.to/torrent/1124395/Fast-and-Furio...,0
4,http://1337x.to/torrent/1145504/Avengers-Age-o...,0
...,...,...
35373,https://lastpass.com/signup2.php?ac=1&from_uri...,0
35374,https://lastpass.com/signup2.php?ac=1&from_uri...,0
35375,https://lastpass.com/signup2.php?ac=1&from_uri...,0
35376,https://lastpass.com/signup2.php?ac=1&from_uri...,0


In [8]:
with open('./substr集合.txt', 'r', encoding='utf-8') as f:
    substrings=f.readlines()
substrings = [s.strip() for s in substrings]
substring_pattern = '|'.join(substrings)

In [9]:
def contain_substring(url):
    result = re.search(substring_pattern, url)
    if result:
        return True
    return False

In [15]:
new_keywords = []

In [16]:
keywords = ['tracking_id', 'clickid3', 'click_id3',
    'clickid2', 'click_id2', 'click_id', 'clickid', 'irclickid2', 
    'irclick_id', 'irclickid', 'affiliate_transaction', 
    'affiliate_campaign', 'sub_affiliate', 
    'affiliate', 'aff_code', 'subaffiliate', 'affiliate_code', 
    'coupon_code', 'affiliate_url', 'affiliate_ref', 'affiliate_source', 
    'affiliate_partner', 'affiliate_banner', 'affiliate_pub', 'aff', 'affid',
    'affiliateid', 'affiliate_id', 'affiliate_link', 'affiliate_track', 'aff_id',
    'affiliate_channel', 'af', 'affiliate-code', 'aff_offer_id', 'fpm-affiliate-utm-source',
    'aff_ref', 'isAffiliate', 'affExt', 'aw_affiliate', 'sub_aff_id',
    'subAffId', 'affiliate_banner_id', 'epic_affiliate', 'aff_trace_key',
    'affExtPa', 'aff_network', 'aff_request_id', 'affiliateNetwork',
    'SubAffiliateID', 'PAffiliateID', 'aff_sub_id', 'aff_name', 'aff_klick_id',
    'affiliateRef','aff_sub', 'super_affiliate_code', 'aff_pid',
    'aff_short_key', 'aff%5C_id', 'aff_expired_f', 'affiliateN', 'subAffiliateID',
    'aff_click_id', 'aff_adid', 'aff_platform','AffiliateReferenceID', 'aff_sub4',
    'AffiliateCode','aff_source', 'is_affiliate','aff_key','id_affiliate','aff_fcid',
    'cf_affiliate_id','affiliateCode','FpAffiliate','2p_affiliate_id'
]

In [35]:
def parameter_check(url):
    parsed_url = urlparse(url)
    # 获取URL的参数信息
    query = parsed_url.query
    params_kv = re.split(r"[\?&;]+", query)
    try:
        params_kv.remove('')
    except:
        pass
    # print(params_kv)
    for kv in params_kv:
        try:
            k = kv.split('=')[0]
            v = kv.split('=')[1]
            if k.lower() in keywords:
                # print(k)
                return True
            for part in k.split('_'):
                for w in re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?=[A-Z]|$)', part):
                    if w.lower() == 'aff' or w.lower() == 'affiliate':
                        new_keywords.append(k)
                        # print(k)
                        return True
            for part in v.split('_'):
                # print(part)
                for w in re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?=[A-Z]|$)', part):
                    print(w)
                    if w.lower() == 'aff' or w.lower() == 'affiliate':
                        print(part)
                        return True
        except:
            pass

    return False

In [ ]:
def is_known_pattern(url):
    re_admitad = re.compile('(ad.admitad.com/g/)|(ad.admitad.com/goto/)')
    res = re_admitad.findall(url)
    if res:
        return True, 'admitad'

    re_affiliaXe=re.compile('(performance.affiliaxe.com/.*\\?aff_id=)|(performance.affiliaxe.com/.*&aff_id=)')
    res = re_affiliaXe.findall(url)
    if res:
        return True, 'affiliaXe'

    re_AliExpress=re.compile('(s.aliexpress.com/.*\\?af=)|(s.aliexpress.com/.*&af=)')
    res = re_AliExpress.findall(url)
    if res:
        return True, 'AliExpress'
    
    re_Amazon=re.compile('(amazon.com/.*\\?tag=)|(amazon.com/.*&tag=)|(amazon.de/.*\\?tag=)|(amazon.de/.*&tag=)|(amazon.it/.*\\?tag=)|(amazon.it/.*&tag=)|(amazon.in/.*\\?tag=)|(amazon.in/.*&tag=)|(amazon.fr/.*\\?tag=)|(amazon.fr/.*&tag=)')
    res = re_Amazon.findall(url)
    if res:
        return True, 'Amazon'

    re_AmazonPrimeVideo=re.compile('(primevideo.com/.*\\?ref=)|(primevideo.com/.*&ref=)')
    res = re_AmazonPrimeVideo.findall(url)
    if res:
        return True, 'Amazon Prime Video'
    
    re_Apple=re.compile('(itunes.apple.com/.*\\?at=)|(itunes.apple.com/.*&at=)|(apple.com/.*\\?afid=)|(apple.com/.*&afid=)')
    res = re_Apple.findall(url)
    if res:
        return True, 'Apple'
    
    re_Audiobooks=re.compile('(affiliates.audiobooks.com/.*\\?a_aid=.*&a_bid=)|(affiliates.audiobooks.com/.*\\?a_bid=.*&a_aid=)|(affiliates.audiobooks.com/.*&a_bid=.*&a_aid=)')
    res = re_Audiobooks.findall(url)
    if res:
        return True, 'Audiobooks'
    
    re_AvantLink=re.compile('(avantlink.com/.*\\?pw=)|(avantlink.com/.*&pw=)')
    res = re_AvantLink.findall(url)
    if res:
        return True, 'AvantLink'
    
    re_Avangate=re.compile('(secure.avangate.com/.*\\?affiliate=)|(secure.avangate.com/.*&affiliate=)')
    res = re_Avangate.findall(url)
    if res:
        return True, 'Avangate'
    
    re_Awin=re.compile('(awin1.com/.*\\?awinaffid=)|(awin1.com/.*&awinaffid=)|(ad.zanox.com/ppc^)|(zenaps.com/rclick.php\\?)')
    res = re_Awin.findall(url)
    if res:
        return True, 'Awin'
    
    re_Banggood=re.compile('(banggood.com/.*\\?p=)|(banggood.com/.*&p=)')
    res = re_Banggood.findall(url)
    if res:
        return True, 'Banggood'

    re_BookDepository=re.compile('(bookdepository.com/.*\\?a_aid=)|(bookdepository.com/.*&a_aid=)')
    res = re_BookDepository.findall(url)
    if res:
        return True, 'Book Depository'
    
    re_Bookingcom =re.compile('(booking.com/.*\\?aid=)|(booking.com/.*&aid=)')
    res = re_Bookingcom.findall(url)
    if res:
        return True, 'Booking.com'
    
    re_Clickbank =re.compile('(hop.clickbank.net^)')
    res = re_Clickbank.findall(url)
    if res:
        return True, 'Clickbank'
    
    re_CJAffiliate =re.compile('(anrdoezrs.net/click-)|(cj.dotomi.com^)|(dpbolvw.net/click-)|(emjcd.com^)|(jdoqocy.com/click-)|(kqzyfj.com/click-)|(qksrv.net^)|(tkqlhce.com/click-)')
    res = re_CJAffiliate.findall(url)
    if res:
        return True, 'CJ Affiliate'
    
    re_DesignModo =re.compile('(designmodo.com/\\?u=)')
    res = re_DesignModo.findall(url)
    if res:
        return True, 'DesignModo'

    re_Ebay =re.compile('(rover.ebay.com/.*\\?campid=)|(rover.ebay.com/.*&campid=)')
    res = re_Ebay.findall(url)
    if res:
        return True, 'Ebay'
    
    re_Envato =re.compile('(audiojungle.net/.*\\?ref=)|(audiojungle.net/.*&ref=)|(codecanyon.net/.*\\?ref=)|(codecanyon.net/.*&ref=)|(marketplace.envato.com/.*\\?ref=)|(marketplace.envato.com/.*&ref=)|(graphicriver.net/.*\\?ref=)|(graphicriver.net/.*&ref=)|(themeforest.net/.*\\?ref=)|(themeforest.net/.*&ref=)|(videohive.net/.*\\?ref=)|(videohive.net/.*&ref=)')
    res = re_Envato.findall(url)
    if res:
        return True, 'Envato'
    
    re_eCommercePartnersNetwork =re.compile('(buyeasy.by/cashback/)|(buyeasy.by/redirect/)')
    res = re_eCommercePartnersNetwork.findall(url)
    if res:
        return True, 'e-Commerce Partners Network'

    re_Flipkart =re.compile('(flipkart.com/.*\\?affid=)|(flipkart.com/.*&affid=)')
    res = re_Flipkart.findall(url)
    if res:
        return True, 'Flipkart'

    re_GTOmegaRacing =re.compile('(gtomegaracing.com/.*\\?tracking=)|(gtomegaracing.com/.*&tracking=)')
    res = re_GTOmegaRacing.findall(url)
    if res:
        return True, 'GT Omega Racing'
    
    re_Hotellook =re.compile('(search.hotellook.com/.*\\?marker=)|(search.hotellook.com/.*&marker=)')
    res = re_Hotellook.findall(url)
    if res:
        return True, 'Hotellook'
    
    re_Hotmart =re.compile('(hotmart.net.br/.*\\?a=)|(hotmart.net.br/.*&a=)')
    res = re_Hotmart.findall(url)
    if res:
        return True, 'Hotmart'

    re_ImpactRadius =re.compile('(7eer.net/c/)|(evyy.net/c/)')
    res = re_ImpactRadius.findall(url)
    if res:
        return True, 'Impact Radius'
    
    re_KontrolFreek =re.compile('(kontrolfreek.com/.*\\?a_aid=)|(kontrolfreek.com/.*&a_aid=)')
    res = re_KontrolFreek.findall(url)
    if res:
        return True, 'KontrolFreek'

    re_Ladbrokes =re.compile('(online.ladbrokes.com/promoRedirect\\?key=)|(online.ladbrokes.com/promoRedirect\\?.*&key=)')
    res = re_Ladbrokes.findall(url)
    if res:
        return True, 'Ladbrokes'

    re_MakeupGeek =re.compile('(makeupgeek.com/.*\\?acc=)|(makeupgeek.com/.*&acc=)')
    res = re_MakeupGeek.findall(url)
    if res:
        return True, 'Makeup Geek'
    
    re_PepperjamNetwork =re.compile('(gopjn.com/t/)|(pjatr.com/t/)|(pjtra.com/t/)|(pntra.com/t/)|(pntrac.com/t/)|(pntrs.com/t/)')
    res = re_PepperjamNetwork.findall(url)
    if res:
        return True, 'Pepperjam Network'
    
    re_RakutenMarketing =re.compile('(click.linksynergy.com/.*\\?id=)|(click.linksynergy.com/.*&id=)')
    res = re_RakutenMarketing.findall(url)
    if res:
        return True, 'Rakuten Marketing'
    
    re_Skimlinks =re.compile('(go.redirectingat.com/.*\\?id=)|(go.redirectingat.com/.*&id=)')
    res = re_Skimlinks.findall(url)
    if res:
        return True, 'Skimlinks'
    
    re_Smartex =re.compile('(olymptrade.com/.*\\?affiliate_id=)|(olymptrade.com/.*&affiliate_id=)')
    res = re_Smartex.findall(url)
    if res:
        return True, 'Smartex'
    
    re_RewardStyle =re.compile('(rstyle.me^)')
    res = re_RewardStyle.findall(url)
    if res:
        return True, 'RewardStyle'

    re_ShopStyle =re.compile('(shopstyle.it^)')
    res = re_ShopStyle.findall(url)
    if res:
        return True, 'ShopStyle'
    
    re_ShareASale =re.compile('(shareasale.com/r.cfm^)|(shareasale.com/m-pr.cfm^)|(shareasale.com/u.cfm^)')
    res = re_ShareASale.findall(url)
    if res:
        return True, 'ShareASale'
    
    re_Studybay =re.compile('(apessay.com/.*\\?rid=)|(apessay.com/.*&rid=)')
    res = re_Studybay.findall(url)
    if res:
        return True, 'Studybay'
    
    re_TataCLiQ =re.compile('(tatacliq.com/.*\\?cid=af:)|(tatacliq.com/.*&cid=af:)')
    res = re_TataCLiQ.findall(url)
    if res:
        return True, 'TataCLiQ'
    
    re_Thermoworks =re.compile('(thermoworks.com/.*\\?tw=)|(thermoworks.com/.*&tw=)')
    res = re_Thermoworks.findall(url)
    if res:
        return True, 'Thermoworks'
    
    re_Zaful =re.compile('(zaful.com/.*\\?lkid=)|(zaful.com/.*&lkid=)')
    res = re_Zaful.findall(url)
    if res:
        return True, 'Zaful'

    return False, ''

In [18]:
def aff_url_identify(url):
    is_know_patt, patt = is_known_pattern(url)
    if is_know_patt:
        rerturn True
    if contain_substring(url):
        return True
    if parameter_check(url):
        return True
    return False

In [19]:
for i in tqdm(range(len(df))):
    if aff_url_identify(df.at[i, 'url']):
        df.at[i, 'is_affiliate'] = 1

100%|██████████| 35378/35378 [00:02<00:00, 17400.10it/s]


In [22]:
df

,url,is_affiliate
0,http://1337x.to/torrent/1048648/American-Snipe...,0
1,http://1337x.to/torrent/1110018/Blackhat-2015-...,0
2,http://1337x.to/torrent/1122940/Blackhat-2015-...,0
3,http://1337x.to/torrent/1124395/Fast-and-Furio...,0
4,http://1337x.to/torrent/1145504/Avengers-Age-o...,0
...,...,...
35373,https://lastpass.com/signup2.php?ac=1&from_uri...,0
35374,https://lastpass.com/signup2.php?ac=1&from_uri...,0
35375,https://lastpass.com/signup2.php?ac=1&from_uri...,0
35376,https://lastpass.com/signup2.php?ac=1&from_uri...,0


In [23]:
df[df['is_affiliate']==1]

,url,is_affiliate
15836,http://ecnavi.jp/redirect/?url=http://px.a8.ne...,1


In [25]:
df.at[15836, 'url']

'http://ecnavi.jp/redirect/?url=http://px.a8.net/svt/ejp?a8mat=16AFFZ+2XDQUQ+M7Q+6BU5T&id1=%user_id%'

In [38]:
re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?=[A-Z]|$)', '16AFFZ+2XDQUQ+M7Q+6BU5T')

['AFF', 'XDQU', 'B', 'T']

In [36]:
parameter_check(df.at[15836, 'url'])

http
px
a
net
svt
ejp
AFF
16AFFZ+2XDQUQ+M7Q+6BU5T


True

In [12]:
for i in tqdm(range(len(df))):
    urls = []
    urls.append(df.at[i, 'original_url'])
    urls.append(df.at[i, 'final_url'])
    try:
        urls += literal_eval(df.at[i,'redirect_urls'])
    except:
        pass
    for url in urls:
        if aff_url_identify(url):
            df.at[i, 'is_affiliate'] = 1
            break

  0%|          | 720/753238 [00:00<01:44, 7182.61it/s]

100%|██████████| 753238/753238 [01:22<00:00, 9150.70it/s] 


In [20]:
new_keywords = list(set(new_keywords))

In [21]:
new_keywords

[]

In [19]:
df.to_csv('./data_all/urls-with-redirection-labeled.csv', index=False)

In [21]:
df[df['is_affiliate']==1]

,original_url,redirect_urls,final_url,source,is_affiliate
0,https://alii.pub/6o3dam,['https://alii.pub/6o3dam'\n 'https://allshops...,https://www.aliexpress.com/item/10050041493828...,YouTube Search,1
2,http://rstyle.me/n/b9i9d29vde,['http://rstyle.me/n/b9i9d29vde'],https://rstyle.me/n/b9i9d29vde,YouTube Search,1
3,https://rstyle.me/+IMNF676MkVHNh9aOf9TaEQ,NaN,https://rstyle.me/+IMNF676MkVHNh9aOf9TaEQ,YouTube Search,1
6,https://go.magik.ly/ml/e66t/,NaN,https://go.magik.ly/ml/e66t/,YouTube Search,1
8,https://NordVPN.com/madewithlau,['https://nordvpn.com/madewithlau'\n 'https://...,https://nordvpn.com/creator/entertainment/?cou...,YouTube Search,1
...,...,...,...,...,...
753230,https://t.co/GkEDvPr74X,"['https://t.co/GkEDvPr74X', 'https://shp.ee/9q...",https://shopee.co.id/product/271297865/1779790...,Twitter Search,1
753232,https://t.co/Yn0XMsPO4o,"['https://t.co/Yn0XMsPO4o', 'https://shopee.co...",https://sv.shopee.co.id/referral/homeTrending?...,Twitter Search,1
753233,https://t.co/YJRLLtBAuR,"['https://t.co/YJRLLtBAuR', 'https://shp.ee/44...",https://idgame.shopee.co.id/getvouchers/6725dd...,Twitter Search,1
753234,https://t.co/9ogADdxF7O,"['https://t.co/9ogADdxF7O', 'https://shp.ee/tp...",https://shopee.co.id/product/20480005/23671843...,Twitter Search,1


In [22]:
124462/753238

0.16523595463850735